In [1]:
# for the sentiment analysis 

from textblob import TextBlob,Blobber 

from textblob.sentiments import NaiveBayesAnalyzer

In [2]:
import keys, tweepy
%config Completer.use_jedi = False


auth = tweepy.OAuthHandler(keys.consumer_key,keys.consumer_key_secret)
auth.set_access_token(keys.access_token,keys.access_token_secret)
api = tweepy.API(auth,wait_on_rate_limit=True,wait_on_rate_limit_notify=True)

In [3]:
text1 = 'I am Olaf , and I like warm hugs '
text2 = 'The end of the word is comming , we are all going to die.'


blob1 = TextBlob(text1)
blob2 = TextBlob(text2)
blob1.sentiment
blob2.sentiment

Sentiment(polarity=0.0, subjectivity=0.0)

In [4]:
# create a blobber for memory efficient sentiment analysis 
# 

factory_blobber = Blobber(analyzer = NaiveBayesAnalyzer())

texts = [text1,text2]

for t in texts:
    print(factory_blobber(t).sentiment)
    

Sentiment(classification='pos', p_pos=0.7933639947437581, p_neg=0.2066360052562419)
Sentiment(classification='neg', p_pos=0.43974538345290015, p_neg=0.5602546165470998)


In [5]:
# Add sentiment analsysis to the loop that print out the tweets

# Review this article:
# https://www.earthdatascience.org/courses/use-data-open-source-python/intro-to-apis/twitter-data-in-python/

# Experiment with searching for differnt tweets and using the "Standard search operators"
# https://developer.twitter.com/en/docs/twitter-api/v1/tweets/search/guides/standard-operators

# Collect a number of tweets and create a Pandas df containing the text of those tweets
# https://www.earthdatascience.org/courses/use-data-open-source-python/intro-to-apis/twitter-data-in-python/


In [6]:
query = 'cyberpunk 2077'

tweets = api.search(q = query,count = 5,tweet_mode = 'extended',lang ='pl')


for num ,tweet in enumerate(tweets,start = 1):
    print(f'Tweet number:{num}')
    # show the tweet user name and screen name 
    print(f'Name: {tweet.user.name},Screen Name: {tweet.user.screen_name}')
    # show the actual tweet  
    # i used tweet_mode = 'extended' - so need to use full_text 
    # need to use tweet.text if not using extended 
    print(f'Tweet:{tweet.full_text}')
    # print the sentiment 
    print(factory_blobber(tweet.full_text).sentiment) 
    print('\n***')

Tweet number:1
Name: Vigilant Magazine,Screen Name: VigilantMag
Tweet:Taki mały przedsmak tego co nas czeka w @CyberpunkGame  https://t.co/lKe0xt0iGZ
Sentiment(classification='pos', p_pos=0.5, p_neg=0.5)

***
Tweet number:2
Name: Mark Shane,Screen Name: 930RB
Tweet:RT @jacekcreator: Jak @CyberpunkGame boostuje inne marki, w tym Adidas, Sprite czy Porsche - https://t.co/noUnJYv2j9 - jeśli znacie inne ni…
Sentiment(classification='neg', p_pos=0.25, p_neg=0.749999999999999)

***
Tweet number:3
Name: Jacek Jankowski,Screen Name: jacekcreator
Tweet:Jak @CyberpunkGame boostuje inne marki, w tym Adidas, Sprite czy Porsche - https://t.co/noUnJYv2j9 - jeśli znacie inne niż pod linkiem przykłady współpracy, proszę dajcie znać :)  #Cyberpunk2077
Sentiment(classification='neg', p_pos=0.3461538461538464, p_neg=0.6538461538461543)

***
Tweet number:4
Name: Justyna,Screen Name: FundacjaH
Tweet:@Mad_Radek Cyberpunk 2077, Wiedźmin, streamy i treningi z gamerami Illuminar Gaming, Devils. One i X-KOM AGO

In [7]:
import pandas as pd


name = pd.Series ([tweet.user.name for tweet in tweets])
user_sc_name = pd.Series([tweet.user.screen_name for tweet in tweets])
text =pd.Series( [tweet.full_text for tweet in tweets ])

df = pd.concat([name,user_sc_name,text],axis = 1)
df.columns = ['user_name','screen_name','tweet']
df.iloc[0]['tweet']

'Taki mały przedsmak tego co nas czeka w @CyberpunkGame  https://t.co/lKe0xt0iGZ'

In [8]:
location = [[tweet.user.name,tweet.user.location] for tweet in tweets ]

location_df = pd.DataFrame(data=location, 
                    columns=['user', "location"])
location_df

,user,location
0,Vigilant Magazine,
1,Mark Shane,
2,Jacek Jankowski,"Warsaw, Poland"
3,Justyna,Gdańsk
4,Justyna,Gdańsk


In [9]:
# you can customize your Twitter search queries by following the Twitter API documentation.
# look for two words together and exclude the retweets 

new_search = "climate+change -filter:retweets"

tweets = tweepy.Cursor(api.search,
                   q=new_search,
                   lang="en",
                   since='2018-04-23').items(1000)

all_tweets = [tweet.text for tweet in tweets]
all_tweets[:5]


['Just a few places left for our workshops:\n\nWriting the Wind with @jr_carpenter (4pm Fri 20 Nov)\nWriting for Wellbei… https://t.co/xrTpCfVTEV',
 'if you think about it, greta thunberg is kind of like the yung lean of climate change',
 "@BorisJohnson If i wanted an electric car I would have bought one,I'm doing my bit to help towards climate change e… https://t.co/fd98sOuN1u",
 'Prime Minister Boris Johnson set out an ambitious ten-point plan for a green industrial revolution which will allow… https://t.co/8r7EpLr01W',
 '"Red Cross: Global warming bigger threat than coronavirus" https://t.co/H1WQCWAa9K']